In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

# Movshon lab - OpenEphys Converter
This tutorial follows the step-by-step guide for a [NWB Converter](https://github.com/catalystneuro/nwb-conversion-tools/blob/master/documentation/conversion_tools_structure.md#step-by-step-operations)

In [2]:
from movshon_lab_to_nwb import MovshonOpenEphysNWBConverter
from pynwb import NWBFile, NWBHDF5IO
from nwbwidgets import nwb2widget
from pathlib import Path
import yaml
import pprint

## Step 1 - Converter.get_source_schema()

In [3]:
# Get source_schema
source_schema = MovshonOpenEphysNWBConverter.get_source_schema()
pprint.pprint(source_schema['properties'], width=120)

{'OpenEphysRecordingExtractorInterface': {'additionalProperties': True,
                                          'properties': {'folder_path': {'type': 'string'}},
                                          'required': ['folder_path'],
                                          'type': 'object'}}


## Step 2 - Get user-input source_data that complies to the returned full source_schema

In [4]:
# Source data
base_path = Path('/home/luiz/storage/taufferconsulting/client_ben/project_movshon/movshon_data/oephys/')

source_data = dict(
    OpenEphysRecordingExtractorInterface=dict(
        folder_path=str(base_path),
    ),
#     OpenEphysSortingExtractorInterface=dict()
)

pprint.pprint(source_data, width=120)

{'OpenEphysRecordingExtractorInterface': {'folder_path': '/home/luiz/storage/taufferconsulting/client_ben/project_movshon/movshon_data/oephys'}}


## Step 3 - Instantiate Converter

In [5]:
# Initialize converter
converter = MovshonOpenEphysNWBConverter(source_data=source_data)

print('Data interfaces for this converter:')
pprint.pprint(converter.data_interface_objects, width=120)

Loading Open-Ephys: reading settings...
Decoding data from  openephys  format
Data interfaces for this converter:
{'OpenEphysRecordingExtractorInterface': <nwb_conversion_tools.datainterfaces.openephysdatainterface.OpenEphysRecordingExtractorInterface object at 0x7fadfddf0f28>}


## Step 4 - Converter.get_metadata_schema()

In [6]:
# Get metadata_schema
metadata_schema = converter.get_metadata_schema()
pprint.pprint(metadata_schema, width=120)

{'$id': 'metadata.schema.json',
 '$schema': 'http://json-schema.org/draft-07/schema#',
 'additionalProperties': False,
 'description': 'Schema for the metadata',
 'properties': {'Ecephys': {'Device': {'additionalProperties': False,
                                       'properties': {'description': {'description': 'Description of the device (e.g., '
                                                                                     'model, firmware version, '
                                                                                     'processing software version, '
                                                                                     'etc.)',
                                                                      'type': 'string'},
                                                      'manufacturer': {'description': 'the name of the manufacturer of '
                                                                                      'this device',
            

## Step 5 - Automatically fetches available metadata with Converter.get_metadata()

In [7]:
# Get metadata from source data
metadata = converter.get_metadata()
pprint.pprint(metadata, width=120)

{'NWBFile': {'identifier': '98520adc-7076-11eb-ab7c-3b0d1d1018db',
             'session_description': 'no description',
             'session_start_time': datetime.datetime(2017, 10, 7, 20, 43, 23, tzinfo=<StaticTzInfo 'EST'>)}}


## Step 6 - Get user-input metadata

In [9]:
metadata['NWBFile']['session_description'] = 'example conversion'
pprint.pprint(metadata, width=120)

{'NWBFile': {'identifier': '98520adc-7076-11eb-ab7c-3b0d1d1018db',
             'session_description': 'example conversion',
             'session_start_time': datetime.datetime(2017, 10, 7, 20, 43, 23, tzinfo=<StaticTzInfo 'EST'>)}}


## Step 7 - Converter.get_conversion_options_schema()

In [13]:
conversion_options_schema = converter.get_conversion_options_schema()

print('Conversion options for each data interface: \n')
pprint.pprint(conversion_options_schema['properties'], width=120)

Conversion options for each data interface: 

{'OpenEphysRecordingExtractorInterface': {'additionalProperties': False,
                                          'properties': {'overwrite': {'default': False, 'type': 'boolean'},
                                                         'return_scaled': {'default': False, 'type': 'boolean'},
                                                         'save_path': {'type': 'string'},
                                                         'stub_test': {'default': False, 'type': 'boolean'},
                                                         'use_timestamps': {'default': False, 'type': 'boolean'},
                                                         'write_as_lfp': {'default': False, 'type': 'boolean'}},
                                          'required': [],
                                          'type': 'object'}}


## Step 8 - Get user-input conversion options

In [18]:
conversion_options = dict(
    OpenEphysRecordingExtractorInterface=dict(
        write_as_lfp=True
    )
)

## Step 9 - Run conversion user filled metadata and conversion_options

In [20]:
output_file = 'out_example.nwb'

converter.run_conversion(
    metadata=metadata, 
    nwbfile_path=output_file, 
    save_to_file=True,
    conversion_options=conversion_options
)

TypeError: write_recording() got an unexpected keyword argument 'return_scaled'

## Final 1 - Check NWB file

In [ ]:
# load file
fname = 'out_example.nwb'
with NWBHDF5IO(fname, 'r') as io:
    nwbfile = io.read()
    print(nwbfile)

## Final 2 - Check NWB file with widgets

In [ ]:
io = NWBHDF5IO(fname, 'r')
nwbfile = io.read()
nwb2widget(nwbfile)